## Exercise 4.1 (4 Points)

### Epipolar Geometry

1. In your own words, explain the following terms:
   - Epipole
   - Epipolar line
   - Epipolar plane


2.  Explain what the epipolar constraint is. Why does a point in one image have a corresponding geometric relationship with the epipolar line in the other view?


3.  In simple terms, describe the role of the fundamental matrix **F**. Why is it important in epipolar geometry, and how does it relate two camera views?


### Answer
Given object `X` and 2 camera `o1` and `o2`. Call the Image plane 1, 2 as the image plane of camera `o1` and `o2` respectiveline. Denote `x1` and `x2` as the image of object `X` on Image plane 1 and 2 respectively.

1.

- Epipoles `e1` and `e2` are image of a camera on image plane of another camera.
- Epipolar plane `pi` is the plane contains 2 cameras and object `X`
- Epipolar lines `l1` and `l2` are the intersection of Epipolar plane with each of image plane.

2. Using the notations above, we define epipolar contraint is that the corresponding point `x'` of image `x1` must lie on the epipolar line `l2` and vice versa.

In the ray `o1X`, all points will result the image `x1` on the Image plane 1. However, each of these points will result different images on Image plane 2. So, the image `x1` relates to the whole epipolar line `l2`.

3. Fundamental matrix `F` shows the relationship between 2 images of the same object in 2 different Image plane. Fundamental matrix is usefull because it enables the full reconstruction. Fundamental matrix includes not only the extrinsic info of the stereo system but also the intrinsic info of each of 2 cameras.

## Exercise 4.2 (4 Points)

### 8-Point Algorithm

In this task you are given two images ("Corridor1.jpg" and "Corridor2.jpg"). 
1) Detect and match features from those images using OpenCv's SIFT detector and OpenCV's BFFMatcher (https://docs.opencv.org/3.4/da/df5/tutorial_py_sift_intro.html and https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html)

2) Implement the 8-point algorithm and compute the fundamental matrix. Normalize the 2d points before running the 8-point algorithm. 

3) Validate your results by drawing epipolar lines on image 2 and verify that corresponding points lie on these lines. Explain why this happens.

You can find more resources on epipolar geometry in the course book “Multiple View Geometry in computer vision” by R. Hartley and A. Zisserman.

In [49]:
#1)
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the images
img1 = cv2.imread('Corridor1.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Corridor2.jpg', cv2.IMREAD_GRAYSCALE)

# Form matrix T
h, w = img1.shape

T = np.float32([
    [2 / h, 0,      -1],
    [0,     2 / w,  -1],
    [0,     0,      1]
])

#########################
# Detect matching feature
#########################
sift = cv2.SIFT_create()

kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2,k=2)

pairs = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        # good.append(m)
        img1_idx = m.queryIdx
        img2_idx = m.trainIdx

        y1, x1 = kp1[img1_idx].pt
        y2, x2 = kp2[img2_idx].pt

        point1, point2 = np.float32([y1, x1, 1]), np.float32([y2, x2, 1])
        point1, point2 = T @ point1, T @ point2

        pairs.append((point1, point2))

#########################
# 8-point algorithm
#########################
# Build constraint matrix
rows = []
for (p1, p2) in pairs:
    rows.append([
        p1[0] * p2[0],
        p1[1] * p2[0],
        p2[0],
        p1[0] * p2[1],
        p1[1] * p2[1],
        p2[1],
        p1[0],
        p1[1],
        1
    ])

A = np.float32(rows)

U, D, VT = np.linalg.svd(A, full_matrices=False)
F = VT[:, -1].reshape(3, 3).T
U, D, VT =  np.linalg.svd(F, full_matrices=False)
D_ = np.diag([D[1], D[2], 0])
F = U @ D_ @ VT.T
F = np.transpose(T) @ F @ T

In [50]:
F

array([[-1.84516674e-07,  1.57351588e-09,  8.94625187e-05],
       [ 5.50990318e-08,  4.36262084e-09, -3.04410975e-05],
       [ 8.97397319e-05, -6.67212964e-06, -3.89552083e-02]])

In [52]:
l_ = F @ np.float32([y1, x1, 1])
l_

array([-8.67279019e-05,  2.29136629e-05,  4.58280207e-02])

In [55]:
np.float32([y2, x2, 1]).T @ F @ np.float32([y1, x1, 1])

np.float64(0.0271100124156583)